This jupyter notebook is to show function `babs-init`.

In [1]:
# See `README_hotoru _show_babs_init_ipynb.md` for how to run this file!

# See another file how to set up before rendering html: 
#   `notebooks/show_babs_init_InputBIDS.ipynb`

# $ jupyter nbconvert --execute --to html notebooks/show_babs_init_fpfsin.ipynb

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs
 
# import sys
import os
import os.path as op

There are several arguments when calling `babs-init`:

In [10]:
# This notebook only accepts fmriprep with freesurfer ingressed
# ++++++++++++++++++++++++++++++++++
bidsapp = "fmriprep"
bidsapp_version_dash = "20-2-3"
# fmriprep: '20-2-3'; '23-1-3'
task_name = "ingressed-fs-sloppy"

input_ds_name_fs = "freesurfer"
# zip files from anat workflow:
# e.g., "fmriprep_anat" for fmriprep 23.1.3; "freesurfer" for fmriprep 20.2.3

type_session = "single-ses"
which_dataset = "HBN"    # "toy_fake", "toy_real", "CCNP", "HBN"
input_ds_fs = "/cbica/projects/BABS/data/HBN_fmriprep_anat"
# ^^ path to the fmriprep `--anat-only` cloned output ria 

flag_where = "cubic"     # "cubic" or "local" or "msi"
type_system = "sge"   # "sge" or "slurm"
# ++++++++++++++++++++++++++++++++++

# sanity checks:
if flag_where == "cubic":
    assert type_system == "sge"
elif flag_where == "msi":
    assert type_system == "slurm"

# where:
if flag_where == "cubic":
    where_root = "/cbica/projects/BABS"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "local":
    where_root = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "msi":
    where_root = "/home/faird/zhaoc"
    where_project = "/home/faird/zhaoc/data"
    where_notebooks = op.join(where_root, "babs/notebooks")
else:
    raise Exception("not valid `flag_where`!")

# Input dataset:
if which_dataset == "CCNP":
    assert type_session == "single-ses"
    input_ds_bids = "/home/faird/zhaoc/data/CCNP_BIDS"   # cloned from RBC github org
elif which_dataset == "HBN":
    assert type_session == "single-ses"
    input_ds_bids = "/cbica/projects/BABS/data/HBN_BIDS"
elif which_dataset == "toy_fake":
    if type_session == "multi-ses":
        input_ds_bids = op.join(where_project, "w2nu3")  # bids, multi-ses
        input_ds_fs = op.join(where_project, "k9zw2")   # fmriprep done, multi-ses
    elif type_session == "single-ses":
        input_ds_bids = op.join(where_project, "t8urc")   # bids, single-ses
        input_ds_fs = "osf://2jvub/"   # fmriprep done, single-ses
elif which_dataset == "toy_real":
    if type_session == "multi-ses":
        # input_ds_bids = "/cbica/projects/RBC/chenying_practice/data_for_babs/NKI/data_hashedID_bids"
        input_ds_bids = "/cbica/projects/BABS/data/testdata_NKI/data_hashedID_bids"
    elif type_session == "single-ses":
        raise Exception("not supported yet!")
else:
    raise Exception("not valid `which_dataset`!")

assert "ingressed-fs" in task_name

bidsapp = "fmriprep"
container_name = "fmriprep" + "-" + bidsapp_version_dash

project_name = "test_babs_" + type_session
# Based on which dataset:
if which_dataset in ["HBN", "PNC", "CCNP", "customize"]:   # specific real dataset
    project_name += "_" + which_dataset
project_name += "_" + bidsapp + "-" + bidsapp_version_dash +  "_" + task_name

# Container:
container_ds = op.join(where_project, bidsapp + "-container")  
if flag_where == "local":
    # container_ds += "-docker"   # add "docker" at the end
    container_ds = op.join(where_project, "toybidsapp-container-docker")

container_config_yaml_file = op.join(where_notebooks, "eg_" + container_name
    + "_" + task_name + "_" + type_system)
if flag_where in ["cubic", "msi"]:
    container_config_yaml_file += "_" + flag_where
else:
    if type_system == "sge":
        container_config_yaml_file += "_" + "cubic"
    elif type_system == "slurm":
        container_config_yaml_file += "_" + "msi"
container_config_yaml_file += ".yaml"
print(container_config_yaml_file)
assert op.exists(container_config_yaml_file)

# list_sub_file = op.join(where_notebooks, "initial_sub_list_" + type_session + ".csv")
list_sub_file = None

/cbica/projects/BABS/babs/notebooks/eg_fmriprep-20-2-3_ingressed-fs-sloppy_sge_cubic.yaml


Let's pass these arguments into `babs-init` CLI:

In [11]:
cmd = "babs-init \\\n"
cmd += "\t" + "--where_project " + where_project + " \\\n"
cmd += "\t" + "--project_name " + project_name + " \\\n"
cmd += "\t" + "--input " + "BIDS" + " " + input_ds_bids + " \\\n"
cmd += "\t" + "--input " + input_ds_name_fs + " " + input_ds_fs + " \\\n"
if list_sub_file is not None:
    cmd += "\t" + "--list_sub_file " + list_sub_file + " \\\n"
cmd += "\t" + "--container_ds " + container_ds + " \\\n"
cmd += "\t" + "--container_name " + container_name + " \\\n"
cmd += "\t" + "--container_config_yaml_file " + container_config_yaml_file + " \\\n"
cmd += "\t" + "--type_session " + type_session + " \\\n"
cmd += "\t" + "--type_system " + type_system

print("# The command to execute:")
print(cmd)
print("WARNING: make sure you've changed `--fs-license-file` value in YAML file if you use it!!!")

# The command to execute:
babs-init \
	--where_project /cbica/projects/BABS/data \
	--project_name test_babs_single-ses_HBN_fmriprep-20-2-3_ingressed-fs-sloppy \
	--input BIDS /cbica/projects/BABS/data/HBN_BIDS \
	--input freesurfer /cbica/projects/BABS/data/HBN_fmriprep_anat \
	--container_ds /cbica/projects/BABS/data/fmriprep-container \
	--container_name fmriprep-20-2-3 \
	--container_config_yaml_file /cbica/projects/BABS/babs/notebooks/eg_fmriprep-20-2-3_ingressed-fs-sloppy_sge_cubic.yaml \
	--type_session single-ses \
	--type_system sge
